## Processamento de Linguagem Natural para Análise de Críticas de Filmes

Database em https://www.kaggle.com/luisfredgs/imdb-ptbr

In [33]:
import pandas as pd

In [34]:
review = pd.read_csv("imdb-reviews-pt-br.csv")
review.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [35]:
# trocando neg por 0 e pos por 1
change = review["sentiment"].replace(["neg", "pos"], [0, 1])
change.head()

0    0
1    0
2    0
3    0
4    0
Name: sentiment, dtype: int64

In [36]:
# adicionando coluna a review
review["sentimentBIN"] = change
review.head()

,id,text_en,text_pt,sentiment,sentimentBIN
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0


In [37]:
# checando se dados estão balanceados para executar treinamento
print(review["sentimentBIN"].value_counts())

0    24765
1    24694
Name: sentimentBIN, dtype: int64


In [38]:
# retirando acentuação
!pip install unidecode
import unidecode

withoutAccentuation = list()

for i in review["text_pt"]: 
    withoutAccentuation.append(unidecode.unidecode(i))
    
review['reviewWithoutAccentuation'] = withoutAccentuation

review['reviewWithoutAccentuation'][0]

'Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessario. Alem das terriveis sequencias de resgate no mar, das quais ha muito poucas, eu simplesmente nao me importei com nenhum dos personagens. A maioria de nos tem fantasmas no armario, e o personagem Costers e realizado logo no inicio, e depois esquecido ate muito mais tarde, quando eu nao me importava. O personagem com o qual deveriamos nos importar e muito arrogante e superconfiante, Ashton Kutcher. O problema e que ele sai como um garoto que pensa que e melhor do que qualquer outra pessoa ao seu redor e nao mostra sinais de um armario desordenado. Seu unico obstaculo parece estar vencendo Costner. Finalmente, quando estamos bem alem do meio do caminho, Costner nos conta sobre os fantasmas dos Kutchers. Somos informados de por que Kutcher e levado a ser o melhor sem pressentimentos ou pressagios anteriores. Nenhuma magica aqui, era tudo que eu podia fazer para nao desligar uma hora.'

In [39]:
# deixando tudo minúsculo
lowercasePhrase = list()

for i in review["reviewWithoutAccentuation"]: 
    lowercasePhrase.append(i.lower())

review['reviewLowercase'] = lowercasePhrase

review['reviewLowercase'][0]

'mais uma vez, o sr. costner arrumou um filme por muito mais tempo do que o necessario. alem das terriveis sequencias de resgate no mar, das quais ha muito poucas, eu simplesmente nao me importei com nenhum dos personagens. a maioria de nos tem fantasmas no armario, e o personagem costers e realizado logo no inicio, e depois esquecido ate muito mais tarde, quando eu nao me importava. o personagem com o qual deveriamos nos importar e muito arrogante e superconfiante, ashton kutcher. o problema e que ele sai como um garoto que pensa que e melhor do que qualquer outra pessoa ao seu redor e nao mostra sinais de um armario desordenado. seu unico obstaculo parece estar vencendo costner. finalmente, quando estamos bem alem do meio do caminho, costner nos conta sobre os fantasmas dos kutchers. somos informados de por que kutcher e levado a ser o melhor sem pressentimentos ou pressagios anteriores. nenhuma magica aqui, era tudo que eu podia fazer para nao desligar uma hora.'

### WORLD CLOUD

Biblioteca em https://github.com/amueller/word_cloud

Para criar a nuvem de palavras é necessário retirar do texto palavras que são irrelevantes (stop words) para que a análise não seja corrompida por um excesso de palavras como "de", "para", "a", "o", etc

In [40]:
# download data
!pip install -U nltk
import nltk
nltk.download('all')

Requirement already up-to-date: nltk in /Users/izabellamelo/miniconda3/lib/python3.7/site-packages (3.4)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/izabellamelo/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/izabellamelo/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/izabellamelo/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/izabellamelo/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/izabellamelo/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/izabellamelo/nltk_data...
[nltk_data]    |   Package cess_

True

In [ ]:
from nltk import tokenize
stopWords = nltk.corpus.stopwords.words("portuguese")
newStopWords = ["filme", "eu", "ser", "sobre", "filmes", "...", "-", "tao", "apenas", 
                "ter", "pode", "outro", "porque", "qualquer", "filme", "pessoa", "cena", 
                "entao", "todo", "todos", "pessoas", "personagens", "vez", "faz", "algumas", "tempo", "/",
                "ver", "ainda", "sido", "the", "vi", "personagem", "vai", "cenas", "tipo", "lo", ".",
                ",", '"', "!", "?", ":", '",', '".', ";", "ano", "realmente", "nao", "voce", "sao", "tambem", "ha",
               "ate", "ja", "so", "aqui", "historia"]
stopWords.extend(newStopWords)
# print(stopWords)
withoutStopWords = list()
spaceToken = tokenize.WordPunctTokenizer()

for i in review["reviewLowercase"]:
    phrase = list()
    textWords = spaceToken.tokenize(i)
    
    for words in textWords:
        if words not in stopWords:
            phrase.append(words)
    
    withoutStopWords.append(' '.join(phrase))

review['reviewWithoutStopWords'] = withoutStopWords

review['reviewWithoutStopWords'][0]
    


Já com o texto sem palavras irrelevantes, podemos exibir uma nuvem de palavras de resenhas positivas e de resenhas
negativas


In [ ]:
!pip install wordcloud
!python -m pip install -U matplotlib

%matplotlib inline

In [ ]:
from wordcloud import WordCloud

# nuvem de palavras positivas
posReview = review.query("sentiment=='pos'")
print(posReview["reviewWithoutStopWords"][12389])

wordsPos = ' '.join([word for word in posReview["reviewWithoutStopWords"]])

In [ ]:
cloudPos = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(wordsPos)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30,7))
plt.imshow(cloudPos, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# nuvem de palavras negativas
negReview = review.query("sentiment=='neg'")
print(negReview["reviewWithoutStopWords"][0])

wordsNeg = ' '.join([word for word in negReview["reviewWithoutStopWords"]])

In [ ]:
cloudNeg = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(wordsNeg)

In [ ]:
plt.figure(figsize=(30,7))
plt.imshow(cloudNeg, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
spaceToken = tokenize.WordPunctTokenizer()
allWords = ' '.join([text for text in review["reviewWithoutStopWords"]])

frequency = nltk.FreqDist(spaceToken.tokenize(allWords))

frequencyDF = pd.DataFrame({"Palavras": list(frequency.keys()),
                           "Frequencia": list(frequency.values())})
frequencyDF.head()

In [ ]:
frequencyDFtop10 = frequencyDF.nlargest(columns="Frequencia", n=30)
frequencyDFtop10

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns

plt.figure(figsize=(12,8))
ax = sns.barplot(data=frequencyDFtop10, x="Palavras", y="Frequencia")
ax.set(ylabel="Contagem")
plt.show()

### BAG OF WORDS
Neste modelo, o texto (uma frase ou documento) é representado como um multiconjunto de suas palavras (o "saco"), desconsiderando a estrutura gramatical e até mesmo a ordenação delas, mas mantendo sua multiplicidade


In [ ]:
# implementando bag of words (fazendo dicionário de palavras)
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
model = CountVectorizer(lowercase=True, max_features=50)

bagOfWords = model.fit_transform(review["reviewWithoutStopWords"])
model.get_feature_names()

In [ ]:
# transformando numa matriz para visualizar melhor
dictionary = pd.SparseDataFrame(bagOfWords, columns = model.get_feature_names())
dictionary.head()

In [ ]:
# separando dados para treino e teste (75% treino, 25% teste)
from sklearn.model_selection import train_test_split

In [ ]:
train, test, trainClass, testClass = train_test_split(bagOfWords, review["sentimentBIN"], random_state = 42)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# treinando
logisticRegression = LogisticRegression(solver='lbfgs')
logisticRegression.fit(train, trainClass)

# testando
score = logisticRegression.score(test, testClass)
print(score)